# ENTRENAMIENTO DEL DATASET UTILIZANDO FASTTEXT.

Del dataset de entrenamiento original, empleamos el 80% del mismo para entrenar usando FastText. Por usuario se arma una oración. La misma se compone de la etiqueta correspondiente al "product_id" del objeto comprado y de las "palabras". Cada palabra representa un evento de dicho usuario y las palabras no son otra cosa que los "product_id" de los objetos vistos (event_type=view). En el caso de los objetos buscados (event_type=search), la palabra correspondiente a cada evento de este tipo es la primera palabra del string de busqueda. El test se hace sobre el 20% del dataset restante. Se calcula el nDCG segun lo indicado en el ML Challenge. 

## Importamos las librerias necesarias y cargamos los datasets.

In [216]:
import fasttext
import gzip
import pandas as pd
import urllib
import tarfile
import urllib.request
import numpy as np
import random
import json
import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import scipy.sparse as sps

url_item_data = "https://meli-data-challenge.s3.amazonaws.com/2020/item_data.jl.gz"
url_train_data = "https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz"
print('Cargando datos de training...')
train_data = []
with urllib.request.urlopen(url_train_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    train_data.append(json.loads(line.strip().decode('utf-8')))


    
train_df = pd.DataFrame(train_data)
train_df.user_history

print('Cargando datos de items...')


item_data = []
with urllib.request.urlopen(url_item_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    item_data.append(json.loads(line.strip().decode('utf-8')))

item_df = pd.DataFrame(item_data)
del train_data
del item_data
del gz
print('Comienza el entrenamiento...')

Cargando datos de training...
Cargando datos de items...
Comienza el entrenamiento...


## Armamos las "oraciones"

Dados los resultados previos donde vimos que más del 95% de los usuarios realizaban sus compras dentro de las 96hs previas al ultimo evento registrado, decidimos quedarnos con los eventos de las ultimas 96hs contando desde el ultimo evento registrado. Armamos entonces las oraciones con esos eventos. 

In [217]:
oracion=[]

for row in train_df.itertuples():
    item_label='__label__'+str(row.item_bought)
    item=''
    #for i in range(0,len(row.user_history)):
    j=0   
    for i in range(len(row.user_history)-1,-1,-1): #del evento final hacia el primer evento
        
        #Calculo los tiempos entre el ultimo evento y cada evento anterior hasta las 96hs. 
        
        end_time=row.user_history[-1]['event_timestamp'][0:23]
        #print(datetime.strptime(end_time,'%Y-%m-%dT%H:%M:%S.%f')
        delta_time=datetime.strptime(end_time, '%Y-%m-%dT%H:%M:%S.%f')-datetime.strptime(row.user_history[i]['event_timestamp'][0:23], '%Y-%m-%dT%H:%M:%S.%f') 
        delta_time=delta_time/pd.Timedelta(hours=1)        
        if delta_time<96 :
           j=j+1 #numero de eventos dentro de las ultimas 96hs. 
    ini_event=len(row.user_history)-j # evento AHORA inicial de cada usuario
    
    for k in range(ini_event,len(row.user_history)): #del NUEVO evento inicial al evento final. 
        if row.user_history[i]['event_type']=='view': # asi accedo x ejemplo a la primera tupla de mi user_history que contiene toda la info de un evento. 
          item=str(((row.user_history)[i]['event_info']))+' '+item # Y asi al valor asociado a una key en particular. 
        else:
           item=str(((row.user_history)[i]['event_info'])).split(sep=' ')[0].lower()+' '+item # Y asi al valor asociado a una key en particular.  
    total_item=item_label+' '+item
    oracion.append(total_item)
    

In [218]:
print('Las oraciones nos quedan, por ejemplo: ', oracion[0])

Las oraciones nos quedan, por ejemplo:  __label__1748830 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 1786148 


## Entrenamos con FastText. 

Para el entrenamiento, luego de haber entrenado con los hiperparametros por default y obtener resultados no muy buenos, decidimos luego de varios intentos, optar por entrenar empleando una tasa de aprendizaje (lr) de 0.5 y utilizar 15 epocas (epochs). Eligiendo estos hiperparametros hemos logrado evitar el overfitting que ocurría con otros valores así como errores debido a tasas de aprendizaje muy altas. 

In [219]:
# Division del dataset.

import numpy as np
from sklearn.model_selection import train_test_split

x_train,x_test=train_test_split(oracion, test_size=0.20, random_state=4)


In [221]:
# Guardamos los dataset de entrenamiento y test en un formato aceptable para FastText

with open('train_file_s&v_80_lr05_epch15_last_96hs.txt', 'w', encoding="utf-8") as f:
    for item in x_train:
        f.write("%s\n" % item)

with open('test_file_s&v_20_lr05_epch15_last_96hs.txt', 'w', encoding="utf-8") as f:
    for item in x_test:
        f.write("%s\n" % item)

In [222]:
# Entrenamiento del modelo con los HP seleccionados. 

model2=fasttext.train_supervised(input='train_file_s&v_80_lr05_epch15_last_96hs.txt',lr=0.5,epoch=15)

In [223]:
# Guardamos el modelo. 

model2.save_model("model_fasttext_80_s&v_lr05_epch15_last_96hs.bin")

In [224]:
# De la misma manera que se entrenó el modelo para las ultimas 96 hs tmb se entrenó el modelo
# con todos los eventos y se guardó. En caso de querer cargar dicho modelo u otros modelos: 

#model2=fasttext.load_model('model_fasttext_80_s&v_lr05_epch15_last_96hs.bin')
model3=fasttext.load_model('model_fasttext_s&v_80_lr05_epch15.bin')


## Testeamos el/los modelo/s

In [225]:
metrics2=model2.test("test_file_s&v_20_lr05_epch15_last_96hs.txt")

print('Para el modelo de las ultimas 96hs, obtenemos que la Precision es de: ', metrics2[1], ' y la Recall es de: ', metrics2[2])

Para el modelo de las ultimas 96hs, obtenemos que la Precision es de:  0.022959513646358815  y la Recall es de:  0.022959513646358815


In [226]:
# El modelo que incluye todos los eventos es testeado con un conjunto de test que incluye todos los eventos y que
# fue creado en una instancia anterior. 

metrics3=model3.test("test_file_s&v_20_lr05_epch15.txt") 

print('Para el modelo c/ todos los eventos, obtenemos que la Precision es de: ', metrics3[1], ' y la Recall es de: ', metrics3[2])

Para el modelo c/ todos los eventos, obtenemos que la Precision es de:  0.09639115250291036  y la Recall es de:  0.09639115250291036


Observamos que la precision y la recall resultan mucho mejor con el modelo que incluye todos los eventos. Sin embargo, es necesario chequear tmb la metrica nDCG para asegurarnos que no haya overfitting. 

Por otro lado, otros modelos fueron entrenados con anterioridad. A continuacion, se presentan las metricas de dichos intentos anteriores. 
* Empleando los HP x default e ignorando los eventos del tipo Search:

Se obtenían una precision y una recall de P=0.01389, R=0.01389 

* Igual al caso anterior pero incluyendo los eventos del tipo Search: 

Mejoraron las metricas mencionadas, alcanzando ambas un valor de: 0.01399

* Ahora, modificando los parametros por default a los finalmente elegidos (lr=0.5, epochs=15), se obtuvieron los siguientes valores: 

p= 0.09693441986806364, r=0.09693441986806364. 

Modificar los HP mejoró las metricas en casi un 700%. 

DECIDIMOS QUEDARNOS CON ESTOS HP. 



## Calculo de nDCG

### Modelo con todos los eventos.

El calculo de nDCG para todos los eventos debe hacerse con los datasets de entrenamiento y de test que incluyan a todos los eventos. Este calculo fue realizado con anterioridad (de la misma manera que se verá a continuación) y los valores resultantes fueron:

nDCG obtenido en datos de ENTRENAMIENTO: 0.7601697277065944
nDCG obtenido en datos de TEST: 0.14587328260284207

Luego, el modelo está overfitteando al conjunto de entrenamiento, de allí un valor de nDCG tan alto que no se reproduce en el conjunto de test. 





### Modelo con las ultimas 96hs.
Se calcula nDCG para los datasets de entrenamiento y de test. 

In [ ]:

item_list = item_df['item_id'].tolist()
domain_list = item_df['domain_id'].tolist()
item_domain_dict = dict(zip(item_list, domain_list))

true_bitem = []
pred_y_fastt_model = []

for row in x_train:
  tb = int(row[:20].split(' ')[0][9:])
  true_bitem.append(tb)

  preds = []
  model_preds = model2.predict(row.split(' ',1)[1],k=10)[0]
  for pred in model_preds:
    preds.append(int(pred[9:]))
  pred_y_fastt_model.append(preds)

#CALCULO DEL nDCG
def ndcg(predictions,bitem):
  ''' Dada una List de predictions, y un item comprado (int), devuelve el calculo de nDCG de la prediccion'''
  best = 22.4246159748234
  current = 0
  bdom = item_domain_dict[bitem]
  for i in range(1,11):
    pred = predictions[i-1]
    if pred == bitem:
      gain = 12
    elif item_domain_dict[pred] == bdom:
      gain = 1
    else:
      gain = 0
    current += (1/np.log(1+i)) * gain
  return current/best 

# true_bitem = train_df.item_bought.tolist()
ndcg_list = []
for i in range(len(true_bitem)):
  ndcg_list.append(ndcg(pred_y_fastt_model[i], true_bitem[i]))

print('nDCG obtenido en datos de ENTRENAMIENTO:' , sum(ndcg_list) / len(ndcg_list))


true_bitem = []
pred_y_fastt_model = []

for row in x_test:
  tb = int(row[:20].split(' ')[0][9:])
  true_bitem.append(tb)

  preds = []
  model_preds = model2.predict(row.split(' ',1)[1],k=10)[0]
  for pred in model_preds:
    preds.append(int(pred[9:]))
  pred_y_fastt_model.append(preds)

#CALCULO DEL nDCG
def ndcg(predictions,bitem):
  ''' Dada una List de predictions, y un item comprado (int), devuelve el calculo de nDCG de la prediccion'''
  best = 22.4246159748234
  current = 0
  bdom = item_domain_dict[bitem]
  for i in range(1,11):
    pred = predictions[i-1]
    if pred == bitem:
      gain = 12
    elif item_domain_dict[pred] == bdom:
      gain = 1
    else:
      gain = 0
    current += (1/np.log(1+i)) * gain
  return current/best 

# true_bitem = train_df.item_bought.tolist()
ndcg_list = []
for i in range(len(true_bitem)):
  ndcg_list.append(ndcg(pred_y_fastt_model[i], true_bitem[i]))

print('nDCG obtenido en datos de TEST:' , sum(ndcg_list) / len(ndcg_list))

nDCG obtenido en datos de ENTRENAMIENTO: 0.4253912812892881
nDCG obtenido en datos de TEST: 0.05485672754797448


Finalmente obtenemos los siguientes valores para nDCG:

* Ultimas 96hs con lr=0.5 y 15 épocas:

nDCG obtenido en datos de ENTRENAMIENTO: 0.43
nDCG obtenido en datos de TEST: 0.05

* Considerando todos los eventos y con lr=0.5 y 15 épocas: 

Para datos de entrenamiento: nDCG = 0.74
Para datos de test: nDCG = 0.14

Luego, los resultados muestran que aunque metricas como la precision y la recall mejoran incluyendo todos los eventos, el uso de estos eventos esta generando un overfitting pues mientras el valor de nDCG mejora considerablemente para los datos de entrenamiento (0.74), no lo hace para los datos de test (0.14). Luego, creemos que lo optimo es trabajar con las ultimas 96hs. 



